In [95]:
import numpy as np
import random
import IPython.display as ipd
import matplotlib.pyplot as plt
import math 

from scipy.interpolate import interp1d
import librosa
import librosa.display
import parselmouth

import seaborn as sns


In [191]:

srate = 44100
#bass, sr = librosa.load('TEST DATA/stems/ANNA MIA - No Need To Be Afraid - Bass.mp3',srate)
#bass = bass[30*sr:60*sr] # 30 seconds of audio

#drums, _ = librosa.load('TEST DATA/stems/ANNA MIA - No Need To Be Afraid - Drums.mp3',srate)
#drums = drums[30*sr:60*sr] # 30 seconds of audio

bass, sr = librosa.load('TEST DATA/stems/bass.wav',srate)
#bass = bass[30*sr:60*sr] # 30 seconds of audio

drums, _ = librosa.load('TEST DATA/stems/drums.wav',srate)
#drums = drums[30*sr:60*sr] # 30 seconds of audio


tempo, beat_times = librosa.beat.beat_track(drums, sr=sr, start_bpm=120, trim=False, units='time') 

clicks = librosa.clicks(beat_times, sr=sr, length=len(bass))

In [190]:
snd = parselmouth.Sound('TEST DATA/stems/bass.wav')
sr = snd.sampling_frequency


In [188]:

data = np.zeros((len(beat_times),2))
for i, beat_time in enumerate(beat_times):
    if i == len(beat_times)-1:
        break
    sd = snd.extract_part(from_time=beat_times[i], to_time = beat_times[i+1], preserve_times=True)
    pitch = sd.to_pitch(pitch_floor = 27.50) # pitch floor is A0
    freq = 0
    count = 0
    for c in pitch.to_array():
        for r in c:
            count+=1
            freq+=r[0]
        break
            
    freq/=count
            
    data[i][0] = freq
    data[i][1] = beat_time
    
    
print(data) # <-------------------- OUTPUT

[[  0.          12.80580499]
 [  0.          13.15410431]
 [  0.          13.49079365]
 [  0.          13.82748299]
 [  0.          14.16417234]
 [  0.          14.50086168]
 [  0.          14.83755102]
 [  0.          15.17424036]
 [  0.          15.51092971]
 [  0.          15.84761905]
 [  0.          16.18430839]
 [  0.          16.52099773]
 [  0.          16.85768707]
 [  0.          17.19437642]
 [  0.          17.54267574]
 [  0.          17.89097506]
 [  0.          18.23927438]
 [  0.          18.5875737 ]
 [  0.          18.93587302]
 [  0.          19.28417234]
 [  0.          19.63247166]
 [  0.          19.98077098]
 [  0.          20.32907029]
 [  0.          20.67736961]
 [  0.          21.02566893]
 [  0.          21.37396825]
 [  0.          21.72226757]
 [  0.          22.07056689]
 [  0.          22.41886621]
 [  0.          22.76716553]
 [  0.          23.11546485]
 [  9.7532635   23.46376417]
 [ 57.64482848  23.81206349]
 [ 69.18265831  24.16036281]
 [ 88.07293065

# IMPORTANT CELL 

In [ ]:
import librosa
import librosa.display
import parselmouth

#load Stem Sample
snd = parselmouth.Sound('TEST DATA/stems/bass.wav')
sr = snd.sampling_frequency 

# Get the beat times from the drum loop
tempo, beat_times = librosa.beat.beat_track(drums, sr=sr, start_bpm=120, trim=False, units='time')

def construct_pitch_track(stem, sr, beat_times):
    data = np.zeros((len(beat_times),2))
    for i, beat_time in enumerate(beat_times):
        if i == len(beat_times)-1:
            break
        sd = snd.extract_part(from_time=beat_times[i], to_time = beat_times[i+1], preserve_times=True)
        pitch = sd.to_pitch(pitch_floor = 27.50) # pitch floor is A0
        freq = 0
        count = 0
        for c in pitch.to_array():
            for r in c:
                count+=1
                freq+=r[0]
            break
            
        freq/=count
            
        data[i][0] = freq
        data[i][1] = beat_time
    
    return data
    

# Random EXTRA code: 

In [189]:
#print(help(parselmouth.Sound))
#print(help(parselmouth.Pitch))
#print(help(parselmouth.Pitch.Candidate))

    
#print(pitch.)

In [ ]:
def draw_pitch(pitch):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")
    
    
    
def draw_spectrogram(spectrogram, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = None
    with np.errstate(divide='ignore'):
        sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

def draw_intensity(intensity):
    plt.plot(intensity.xs(), intensity.values.T, linewidth=3, color='w')
    plt.plot(intensity.xs(), intensity.values.T, linewidth=1)
    plt.grid(False)
    plt.ylim(0)
    plt.ylabel("intensity [dB]")

    
intensity = snd.to_intensity()
spectrogram = snd.to_spectrogram()
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_intensity(intensity)
plt.xlim([snd.xmin, snd.xmax])
plt.show() 


#-------------------------------
#snd = snd.extract_part(from_time=40, to_time = 50, preserve_times=True)

plt.figure(figsize=(14, 5))
plt.plot(snd.xs(), snd.values.T)
plt.vlines(beat_times, -1, 1, color='r')
plt.xlim([snd.xmin, snd.xmax])
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.show() # or plt.savefig("sound.png"), or plt.savefig("sound.pdf")
#print(help(parselmouth.Sound))
#ipd.Audio(snd.values,rate=sr)